In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))




TensorFlow Version: 1.2.1


In [2]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights


    vgg_tag = 'vgg16'
    #vgg_input_tensor_name = 'image_input:0'
    #vgg_keep_prob_tensor_name = 'keep_prob:0'
    #vgg_layer3_out_tensor_name = 'layer3_out:0'
    #vgg_layer4_out_tensor_name = 'layer4_out:0'
    #vgg_layer7_out_tensor_name = 'layer7_out:0'

    tf.saved_model.loader.load(sess,[vgg_tag],vgg_path)

    #image_input = sess.graph.get_tensor_by_name(vgg_input_tensor_name)
    #keep_prob = sess.graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    #layer3_out = sess.graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    #layer4_out = sess.graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    #layer7_out = sess.graph.get_tensor_by_name(vgg_layer7_out_tensor_name)


    return (tf.get_default_graph().get_tensor_by_name( r + ":0") for r in ['image_input','keep_prob','layer3_out','layer4_out','layer7_out'])
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [3]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    num_classes = 2


    #FCN
    layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='SAME')
    layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='SAME')
    layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='SAME')

    # deconv
    deconv = tf.layers.conv2d_transpose(layer7, num_classes, 4, 2, 'SAME')

    # Skip-connection with layer4 and deconv
    comb_layer1 = tf.add(layer4, deconv)

    # deconv
    deconv = tf.layers.conv2d_transpose(comb_layer1, num_classes, 4, 2, 'SAME')

    # Skip-connection with layer3 and deconv
    comb_layer2 = tf.add(layer3, deconv)

    # deconv to original image size(FCN-8s)
    output = tf.layers.conv2d_transpose(comb_layer2, num_classes, 16, 8, 'SAME')

    return output
tests.test_layers(layers)

Tests Passed


In [4]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    #reshape output and label tensor
    logits = tf.reshape(nn_last_layer,(-1,num_classes))
    reshaped_labels = tf.reshape(correct_label, (-1, num_classes))

    #calculate loss
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = reshaped_labels))
    #Optimization
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)
    #GradientDescent is best so far
    return logits,train_op,cross_entropy_loss
tests.test_optimize(optimize)

Tests Passed


In [6]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    idx = 0
    for epoch in range(epochs):
        for batch in get_batches_fn(batch_size):
            idx +=1
            batch_images, batch_gts = batch
            _,loss =sess.run([train_op,cross_entropy_loss] , feed_dict={input_image:batch_images , keep_prob:0.85 , correct_label:batch_gts , learning_rate:0.001})

            #best param
            #_,loss =sess.run([train_op,cross_entropy_loss] , feed_dict={input_image:batch_images , keep_prob:0.85 , correct_label:batch_gts , learning_rate:0.001})

            #print out loss
            if (idx % 2 == 0):
                print("Epoch {}/{}...".format(epoch+1, epochs),
                      "Training Loss: {:.4f}...".format(loss))


tests.test_train_nn(train_nn)

In [ ]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/


    epochs = 10
    batch_size = 16

    # epochs = 10
    #batch_size = 16 is the best so far

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        image_input, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess,vgg_path)
        output = layers(layer3_out,layer4_out,layer7_out,num_classes)

        learning_rate = tf.placeholder(dtype = tf.float32)
        correct_label = tf.placeholder(dtype = tf.float32, shape = (None, None, None, num_classes)) # 4-D tensor, project_tests.py

        logits,train_op,cross_entropy_loss = optimize(output , correct_label , learning_rate , num_classes)
        # TODO: Train NN using the train_nn function
        sess.run(tf.global_variables_initializer())
        train_nn(sess,epochs,batch_size,get_batches_fn,train_op,cross_entropy_loss,image_input,correct_label,keep_prob,learning_rate)
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)


if __name__ == '__main__':
    run()